# Set up EDB for PCB DC IR Analysis
This example shows how to set up the electronics database (EDB) for DC IR analysis from a single
configuration file.

## Perform imports and define constants

Perform required imports.

In [1]:
import json
import os
import tempfile

from ansys.aedt.core import Hfss3dLayout, Icepak
from ansys.aedt.core.examples.downloads import download_file
from pyedb import Edb

Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NG_MODE = False

Download the example PCB data.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")
file_edb = download_file(source="edb/ANSYS-HSD_V1.aedb", local_path=temp_folder.name)

## Load example layout

In [4]:
edbapp = Edb(file_edb, edbversion=AEDT_VERSION)

C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\misc\decorators.py:55: UserWarning: Argument `edbversion` is deprecated for method `Edb`; use `version` instead.
  warnings.warn(
C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\generic\design_types.py:301: UserWarning: Your ANSYS AEDT version is eligible to gRPC version.You might consider switching to that version for better user experience.For more information please check this link: https://edb.docs.pyansys.com/version/dev/grpc_api/index.html
  warnings.warn(GRPC_GENERAL_WARNING, UserWarning)


PyEDB INFO: Star initializing Edb 10:31:22.166238


PyEDB INFO: Edb version 2025.2


PyEDB INFO: Logger is initialized. Log file is saved to C:\Users\ansys\AppData\Local\Temp\pyedb_ansys.log.


PyEDB INFO: legacy v0.67.1


...

PyEDB INFO: Database ANSYS-HSD_V1.aedb Opened in 2025.2


PyEDB INFO: Cell main Opened


PyEDB INFO: Builder was initialized.


PyEDB INFO: open_edb completed in 9.0200 seconds.


PyEDB INFO: EDB initialization completed in 9.0885 seconds.


## Create an empty dictionary to host all configurations

In [5]:
cfg = dict()
cfg["sources"] = []

## Update stackup

In [6]:
cfg["stackup"] = {
    "layers": [
        {"name": "Top", "type": "signal", "material": "copper", "fill_material": "FR4_epoxy", "thickness": "0.035mm"},
        {"name": "DE1", "type": "dielectric", "material": "FR4_epoxy", "fill_material": "", "thickness": "0.1mm"},
        {
            "name": "Inner1",
            "type": "signal",
            "material": "copper",
            "fill_material": "FR4_epoxy",
            "thickness": "0.017mm",
        },
        {"name": "DE2", "type": "dielectric", "material": "FR4_epoxy", "fill_material": "", "thickness": "0.088mm"},
        {
            "name": "Inner2",
            "type": "signal",
            "material": "copper",
            "fill_material": "FR4_epoxy",
            "thickness": "0.017mm",
        },
        {"name": "DE3", "type": "dielectric", "material": "FR4_epoxy", "fill_material": "", "thickness": "0.1mm"},
        {
            "name": "Inner3",
            "type": "signal",
            "material": "copper",
            "fill_material": "FR4_epoxy",
            "thickness": "0.017mm",
        },
        {
            "name": "FR4_epoxy-1mm",
            "type": "dielectric",
            "material": "FR4_epoxy",
            "fill_material": "",
            "thickness": "1mm",
        },
        {
            "name": "Inner4",
            "type": "signal",
            "material": "copper",
            "fill_material": "FR4_epoxy",
            "thickness": "0.017mm",
        },
        {"name": "DE5", "type": "dielectric", "material": "FR4_epoxy", "fill_material": "", "thickness": "0.1mm"},
        {
            "name": "Inner5",
            "type": "signal",
            "material": "copper",
            "fill_material": "FR4_epoxy",
            "thickness": "0.017mm",
        },
        {"name": "DE6", "type": "dielectric", "material": "FR4_epoxy", "fill_material": "", "thickness": "0.088mm"},
        {
            "name": "Inner6",
            "type": "signal",
            "material": "copper",
            "fill_material": "FR4_epoxy",
            "thickness": "0.017mm",
        },
        {"name": "DE7", "type": "dielectric", "material": "FR4_epoxy", "fill_material": "", "thickness": "0.1mm"},
        {
            "name": "Bottom",
            "type": "signal",
            "material": "copper",
            "fill_material": "FR4_epoxy",
            "thickness": "0.035mm",
        },
    ]
}

## Define voltage source

In [7]:
cfg["sources"].append(
    {
        "name": "vrm",
        "reference_designator": "U2",
        "type": "voltage",
        "magnitude": 1,
        "positive_terminal": {"net": "1V0"},
        "negative_terminal": {"net": "GND"},
    }
)

## Define current source

In [8]:
cfg["sources"].append(
    {
        "name": "U1_1V0",
        "reference_designator": "U1",
        "type": "current",
        "magnitude": 10,
        "positive_terminal": {"net": "1V0"},
        "negative_terminal": {"net": "GND"},
    }
)

## Define SIwave DC IR analysis setup

In [9]:
cfg["setups"] = [
    {
        "name": "siwave_1",
        "type": "siwave_dc",
        "dc_slider_position": 1,
        "dc_ir_settings": {"export_dc_thermal_data": True},
    }
]

## Define Cutout

In [10]:
cfg["operations"] = {
    "cutout": {"signal_list": ["1V0"], "reference_list": ["GND"], "extent_type": "ConvexHull", "expansion_size": 0.02}
}

## Define package for thermal analysis (optional)

In [11]:
cfg["package_definitions"] = [
    {
        "name": "package_1",
        "component_definition": "ALTR-FBGA1517-Ansys",
        "maximum_power": 0.5,
        "therm_cond": 2,
        "theta_jb": 3,
        "theta_jc": 4,
        "height": "1mm",
        "apply_to_all": False,
        "components": ["U1"],
    },
]

## Write configuration into a JSON file

In [12]:
file_json = os.path.join(temp_folder.name, "edb_configuration.json")
with open(file_json, "w") as f:
    json.dump(cfg, f, indent=4, ensure_ascii=False)

## Import configuration into example layout

In [13]:
edbapp.configuration.load(config_file=file_json)

Apply configuration to EDB.

In [14]:
edbapp.configuration.run()

PyEDB INFO: Updating nets finished. Time lapse 0:00:00


PyEDB INFO: Updating components finished. Time lapse 0:00:00


PyEDB INFO: Creating pin groups finished. Time lapse 0:00:00


PyEDB INFO: Placing sources finished. Time lapse 0:00:00.727105


PyEDB INFO: Applying materials finished. Time lapse 0:00:00


...

PyEDB INFO: Cutout completed. Elapsed time: 0m 5sec


PyEDB INFO: EDB file save completed in 0.0792 seconds.


PyEDB INFO: Cutout completed in 1 iterations with expansion size of (0.02)*(1000.0)mm Elapsed time: 0m 5sec


PyEDB INFO: Applying operations completed in 5.5016 seconds.


True

Save and close EDB.

In [15]:
edbapp.save()
edbapp.close()

PyEDB INFO: Save Edb file completed in 0.0637 seconds.


PyEDB INFO: Close Edb file completed in 0.1544 seconds.


True

The configured EDB file is saved in a temp folder.

In [16]:
print(temp_folder.name)

C:\Users\ansys\AppData\Local\Temp\tmpcbg8oz7k.ansys


## Load edb into HFSS 3D Layout.

In [17]:
h3d = Hfss3dLayout(edbapp.edbpath, version=AEDT_VERSION, non_graphical=NG_MODE, new_desktop=True)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


PyAEDT INFO: Log on file C:\Users\ansys\AppData\Local\Temp\pyaedt_ansys_7906ce6d-96aa-40ac-99ca-94f2831afdae.log is enabled.


...

PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.


PyAEDT INFO: EDB folder C:\Users\ansys\AppData\Local\Temp\tmpcbg8oz7k.ansys\edb\ANSYS-HSD_V1.aedb has been imported to project ANSYS-HSD_V1


PyAEDT INFO: Active Design set to 0;main


PyAEDT INFO: Active Design set to 0;main


PyAEDT INFO: Aedt Objects correctly read


## Prepare for electro-thermal analysis in Icepak (Optional)

In [18]:
h3d.modeler.set_temperature_dependence(include_temperature_dependence=True, enable_feedback=True, ambient_temp=22)

PyAEDT INFO: Loading Modeler.


PyAEDT INFO: Modeler loaded.


PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Set the temperature dependence for the design.


PyAEDT INFO: Assigned Objects Temperature


True

## Analyze

In [19]:
h3d.analyze()

PyAEDT INFO: Project ANSYS-HSD_V1 Saved correctly


PyAEDT INFO: Solving all design setups. Analysis started...


PyAEDT INFO: Design setup None solved correctly in 0.0h 0.0m 45.0s


True

## Plot DC voltage

In [20]:
voltage = h3d.post.create_fieldplot_layers_nets(
    layers_nets=[
        ["Inner2", "1V0"],
    ],
    quantity="Voltage",
    setup="siwave_1",
)

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpcbg8oz7k.ansys\edb\ANSYS-HSD_V1.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmpcbg8oz7k.ansys\edb\ANSYS-HSD_V1.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.30187320709228516


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


...

C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\generic\design_types.py:301: UserWarning: Your ANSYS AEDT version is eligible to gRPC version.You might consider switching to that version for better user experience.For more information please check this link: https://edb.docs.pyansys.com/version/dev/grpc_api/index.html
  warnings.warn(GRPC_GENERAL_WARNING, UserWarning)


PyEDB INFO: open_edb completed in 0.1748 seconds.


PyEDB INFO: EDB initialization completed in 0.1907 seconds.


PyAEDT INFO: EDB loaded.


PyAEDT INFO: Active Design set to 0;main


In [21]:
file_path_image = os.path.join(temp_folder.name, "voltage.jpg")
voltage.export_image(
    full_path=file_path_image,
    width=640,
    height=480,
    orientation="isometric",
    display_wireframe=True,
    selections=None,
    show_region=True,
    show_axis=True,
    show_grid=True,
    show_ruler=True,
)

'C:\\Users\\ansys\\AppData\\Local\\Temp\\tmpcbg8oz7k.ansys\\voltage.jpg'

## Plot power density

In [22]:
power_density = h3d.post.create_fieldplot_layers_nets(
    layers_nets=[
        ["Inner2", "no-net"],
    ],
    quantity="Power Density",
    setup="siwave_1",
)

PyAEDT INFO: Active Design set to 0;main


In [23]:
file_path_image = os.path.join(temp_folder.name, "power_density.jpg")
power_density.export_image(
    full_path=file_path_image,
    width=640,
    height=480,
    orientation="isometric",
    display_wireframe=True,
    selections=None,
    show_region=True,
    show_axis=True,
    show_grid=True,
    show_ruler=True,
)

'C:\\Users\\ansys\\AppData\\Local\\Temp\\tmpcbg8oz7k.ansys\\power_density.jpg'

## Save HFSS 3D Layout project

In [24]:
h3d.save_project()

PyAEDT INFO: Project ANSYS-HSD_V1 Saved correctly


True

## Create an Icepak design

In [25]:
ipk = Icepak(version=AEDT_VERSION, non_graphical=NG_MODE, new_desktop=False)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Returning found Desktop session with PID 7064!


PyAEDT INFO: No project is defined. Project ANSYS-HSD_V1 exists and has been read.


PyAEDT INFO: No consistent unique design is present. Inserting a new design.


PyAEDT INFO: Added design 'Icepak_QYZ' of type Icepak.


PyAEDT INFO: Aedt Objects correctly read


## Create PCB

In [26]:
pcb = ipk.create_ipk_3dcomponent_pcb(
    compName="PCB_pyAEDT",
    setupLinkInfo=[h3d.project_file, h3d.design_name, "siwave_1", True, True],
    solutionFreq=None,
    resolution=0,
    extent_type="Bounding Box",
    powerin="0",
)

PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Parsing design objects. This operation can take time


PyAEDT INFO: Refreshing bodies from Object Info


PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec


PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec


PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec


## Include pckage definition from Edb

In [27]:
pcb.included_parts = "Device"

## Adjust air region

In [28]:
region = ipk.modeler["Region"]
faces = [f.id for f in region.faces]
ipk.assign_pressure_free_opening(assignment=faces, boundary_name="Outlet")

PyAEDT INFO: Parsing design objects. This operation can take time


PyAEDT INFO: Refreshing bodies from Object Info


PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec


PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec


PyAEDT INFO: Boundary Opening Outlet has been created.


Outlet

## Setup mesh

In [29]:
glob_msh = ipk.mesh.global_mesh_region
glob_msh.global_region.positive_z_padding_type = "Absolute Offset"
glob_msh.global_region.positive_z_padding = "50 mm"
glob_msh.global_region.negative_z_padding_type = "Absolute Offset"
glob_msh.global_region.negative_z_padding = "80 mm"

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpcbg8oz7k.ansys\edb\ANSYS-HSD_V1.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmpcbg8oz7k.ansys\edb\ANSYS-HSD_V1.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.3008768558502197


PyAEDT WARNING: No mesh operation found.


PyAEDT WARNING: No mesh region found.


...

PyAEDT WARNING: Property Command is read-only.


PyAEDT WARNING: Property Command is read-only.


PyAEDT WARNING: Property Command is read-only.


PyAEDT WARNING: Property Command is read-only.


PyAEDT WARNING: Property Command is read-only.


In [30]:
glob_msh = ipk.mesh.global_mesh_region
glob_msh.manual_settings = True
glob_msh.settings["EnableMLM"] = True
glob_msh.settings["EnforeMLMType"] = "2D"
glob_msh.settings["2DMLMType"] = "Auto"
glob_msh.settings["MaxElementSizeY"] = "2mm"
glob_msh.settings["MaxElementSizeX"] = "2mm"
glob_msh.settings["MaxElementSizeZ"] = "3mm"
glob_msh.settings["MaxLevels"] = "2"

In [31]:
glob_msh.update()

True

## Place monitor

In [32]:
cpu = ipk.modeler["PCB_pyAEDT_U1_device"]
m1 = ipk.monitor.assign_face_monitor(
    face_id=cpu.top_face_z.id,
    monitor_quantity="Temperature",
    monitor_name="TemperatureMonitor1",
)

## Create Icepak setup

In [33]:
setup1 = ipk.create_setup(MaxIterations=10)

PyAEDT INFO: Key MaxIterations matched internal key 'Convergence Criteria - Max Iterations' with confidence of 52.


Add 2-way coupling to the setup

In [34]:
ipk.assign_2way_coupling(number_of_iterations=1)

True

## Save

In [35]:
ipk.save_project()

PyAEDT INFO: Project ANSYS-HSD_V1 Saved correctly


True

## Shut Down Electronics Desktop

In [36]:
ipk.release_desktop()

PyAEDT INFO: Desktop has been released and closed.


True

All project files are saved in the folder ``temp_file.dir``. If you've run this example as a Jupyter notebook you
can retrieve those project files.